### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE77563'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)
    

import pandas as pd
from utils.preprocess import geo_select_clinical_features, save_cohort_info, preview_df
import os

# Create directories if not exist
os.makedirs('./preprocessed/Gastroesophageal_reflux_disease_(GERD)/trait_data', exist_ok=True)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = 5
age_row = 1
gender_row = 2

def convert_trait(value):
    try:
        return 1 if value.split(': ')[1].lower() == 'gerd' else 0
    except (IndexError, ValueError, AttributeError):
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (IndexError, ValueError, AttributeError):
        return None

def convert_gender(value):
    try:
        return 1 if value.split(': ')[1].lower() == 'male' else 0
    except (IndexError, ValueError, AttributeError):
        return None

# Save cohort information
save_cohort_info('GSE77563', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', is_gene_available, trait_row is not None)

# Load clinical data (assuming actual data loading code is here)
clinical_data = pd.read_csv('path_to_clinical_data.csv')  # Replace with actual path

# Clinical feature extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Gastroesophageal_reflux_disease_(GERD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/trait_data/GSE77563.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Re-import or redefine required modules and functions first if not available in current context
from utils.preprocess import get_gene_annotation, get_gene_mapping, apply_gene_mapping, preview_df

# Define the soft_file and matrix_file if not already defined
cohort_dir = '../DATA/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE77563'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# Define gene_annotation again to ensure it's within the context
gene_annotation = get_gene_annotation(soft_file)

# Preview the gene_annotation to identify appropriate keys
# print("Gene annotation preview:")
# print(preview_df(gene_annotation))

# Use the preview to determine the appropriate keys:
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'  # This key seems to store gene-related data.

# Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE77563.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Gastroesophageal_reflux_disease_(GERD)')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE77563', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/GSE77563.csv'
    unbiased_merged_data.to_csv(csv_path)
